<img src='images/bildungscampus_logo.png' width="40%" align="left" />
<img src='images/hhn.png' width="25%" align="right" />

# Schritt 6: Chatbot Prototyp
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='images/bar.png'/>

## Zusammenfügen aller Chatbot-Komponenten

Das hier das finale Notebook. Hier werden nun alle Komponenten, die zuvor in den anderen Notebooks entwickelt, trainert und gespeichert wurden, zusammengefügt, um einen voll funktionsfähigen Chatbot zu bilden. 

### 6.1. Import benötigter Bibliotheken

In [1]:
# import of necessary libraries
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn 
import tensorflow
import random 
import json
import pickle
import spacy
import sqlite3
import re
from difflib import SequenceMatcher

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


### 6.2. Intent Classification

Das Neuronale Netzwerk wurde zuvor in einem separaten Notebook erstellt, trainert und gespeichert. Der folgende Code versucht das gespeicherte Model zu laden. Falls dies nicht möglich sein sollte, weil des Model zum Beispiel nicht mehr existiert, wird ein neues Neuronales Netzwerk erstellt und trainiert. Für diesen Fall werden die Trainingsdaten "data.pickle" geladen und die Layer des Netztes definiert.

In [2]:
# load training data 
with open("2. Intent Classififcation/data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

In [3]:
# open saved model or implement new one if model does not exist yet
tensorflow.compat.v1.reset_default_graph()
    
# creating the Neural Network
net = tflearn.input_data(shape= [None, len(training[0])]) #input layer Neurons = numer of words in training
net = tflearn.fully_connected(net, 8) #hidden layer fully connected with 8 neuron
net = tflearn.fully_connected(net, len(output[0]), activation="softmax" ) #output layer 6 Neurons = labels
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load(r"2. Intent Classififcation/model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save(r"2. Intent Classififcation/model.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\Sebi\OneDrive\Studium\Thesis_Chatbot\2. Intent Classififcation\model.tflearn


In [4]:
model

In [5]:
# transform input in numbers to make it readable for neural net
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return numpy.array(bag)

### 6.3. Slot Filling

Für die Extraktion der verschiedenen Zusatzinformationen müssen unteranderem bereits bekannte Namen aus der Datenbank extrahiert werden und in Listen gespeichert werden. Eine Funktion gleicht den Input des Benutzers mit den Namen ab, um zu prüfen ob der vom Benutzer genannte Professor bekannt ist oder nicht. Zusätzlich werden zuvor trainerte Custom NER Modelle geladen und die Slot Filling definiert.

#### 6.3.1. Vor- und Nachnamen

In [6]:
# get first names from database
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select first_name FROM PROF_INFO_TABLE")
conn.commit()
first_names = cur.fetchall()
conn.close()

first_names_list = []
for i in range(len(first_names)):
    answer = " ".join(first_names[i])
    answer.strip()
    first_names_list.append(answer)

#first_names_list

In [7]:
# get last names from database
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select last_name FROM PROF_INFO_TABLE")
conn.commit()
last_names = cur.fetchall()
conn.close()

# store last names in list
last_names_list = []
for i in range(len(last_names)):
    answer = " ".join(last_names[i])
    answer.strip()
    last_names_list.append(answer)

#last_names_list

#### 6.3.3. Forschungsschwerpunkte und Studiengänge

In [8]:
#load the custom NER models
nlp_research_area = spacy.load(r"3. Slot Filling/custom_ner_model_research_area")
nlp_study = spacy.load(r"3. Slot Filling/custom_ner_model_study")

#### 6.3.4. Slot Filling Retrieval Funktion

Wenn der Chatbot, anhand des Intents der Eingabe, erkannt hat, dass Information Retrieval die auszuführende Aktion ist, wird folgende Funktion augeführt. Sie extrahiert die benötigten Informationen aus der Eingabe des Benutzers, um die erfolgreiche Ausführung der Aktion zu gewährleisten. Je nach Intent sind verschiedene Informationen aus der Eingaben zu extrahieren. Für eine bessere Übersichtlichkeit, wurde das Slot Filling nach den auszuführenden Aktionen geteilt. 

In [9]:
def slot_filling_retrieval(inp, intent):   
    # list of intents
    intent_prof_contact=["prof_telephone_query_name", "prof_email_query_name", "prof_office_query_name", "prof_research_area_query_name", "prof_study_query_name"]
    intent_generic_conversation=["greeting","greeting_response","courtesy_greeting","courtesy_greeting_response","real_name_query", "goodbye","task_response"]
        
    # get phone number
    if intent == "prof_name_query_telephone":
        re_number_1 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
        re_number_2 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{3}"
        re_number_3 = r"\D[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
        condition = re.compile("(%s|%s|%s)" % (re_number_1, re_number_2, re_number_3)).findall(inp) 

    # get email 
    elif intent == "prof_name_query_email":
        condition = re.findall('\S+@\S+', inp)

    # get office
    elif intent == "prof_name_query_office":
        re_office_1 = r"[A-Z, a-z].\d{1}.\d{2}"
        re_office_2 = r"[A-Z, a-z].\d{3}"
        condition = re.compile("(%s|%s)" % (re_office_1, re_office_2)).findall(inp)

    # get research area
    elif intent == "prof_name_query_research_area":
        doc = nlp_research_area(inp)
        for ent in doc.ents:
            if ent.label_ == "RESEARCH_AREA":
                condition = ent.text    

    # get study
    elif intent == "prof_name_query_study":
        doc = nlp_study(inp)
        for ent in doc.ents:
            if ent.label_ == "STUDY":
                condition = ent.text  

    # get first name
    elif intent == "prof_name_query_lastname":
        for first_name in first_names_list:
            for word in inp.split():
                if SequenceMatcher(None, first_name, word).ratio() >= 0.7:
                    condition = first_name
                    
    # get last name
    elif intent == "prof_name_query_firstname":
        for last_name in last_names_list:
            for word in inp.split():
                if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                    condition = last_name

    # get last name
    elif intent in intent_prof_contact:
        for last_name in last_names_list:
            for word in inp.split():
                if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                    condition = last_name
    
    # no extra info needed
    elif intent in intent_generic_conversation:
        condition = "generic intent"

    else:
        condition = "nobody/ nothing"
    
    return condition

#### 6.3.5. Slot Filling Update Funktion

Wenn der Chatbot, anhand des Intents der Eingabe, erkannt hat, dass Information Extraction and Storing die auszuführende Aktion ist, wird folgende Funktion augeführt.

In [10]:
def slot_filling_update(inp, intent):
    #get name of professor
    for last_name in last_names_list:
        for word in inp.split():
            if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                prof_name = last_name

    
    # get phone number
    if intent == "extract_new_telephone":
        #get the new number
        re_number_1 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
        re_number_2 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{3}"
        re_number_3 = r"\D[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
        condition = re.compile("(%s|%s|%s)" % (re_number_1, re_number_2, re_number_3)).findall(inp)      
        
    # get email
    elif intent == "extract_new_email":
        condition = re.findall('\S+@\S+', inp)
    
    # get office
    elif intent == "extract_new_office":
        re_office_1 = r"[A-Z, a-z].\d{1}.\d{2}"
        re_office_2 = r"[A-Z, a-z].\d{3}"
        condition = re.compile("(%s|%s)" % (re_office_1, re_office_2)).findall(inp)
        
    # get research area
    elif intent == "extract_new_research_area":
        doc = nlp_research_area(inp)
        for ent in doc.ents:
            if ent.label_ == "RESEARCH_AREA":
                condition = ent.text  
            else:
                condition = "Not found"
    
    # get study
    elif intent == "extract_new_study":
        doc = nlp_study(inp)
        for ent in doc.ents:
            if ent.label_ == "STUDY":
                condition = ent.text  
            else:
                condition = "Not found"
    
    return prof_name, condition

###  6.4. Action Execution 

Nun werden die Funktionen zur Ausführung der jeweiligen Aktionen definiert. Die Aktion Information Extraction and Storing wird in zwei Funktionen aufgeteilt. Eine Funktion kümmert sich um die Spicherung neuer Professoren in die Datenbank und die andere um die Aktualisierung bekannter Informationen.

#### 6.4.1 Action Execution - Information Retrieval

In [11]:
def action_execution_retrieval(intent, condition):
    # list with intents 
    intent_generic_conversation=["greeting","greeting_response","courtesy_greeting","courtesy_greeting_response","real_name_query", "goodbye","task_response"]
    
    #connect to database
    conn = sqlite3.connect("PROF_INFO_DB.db")
    cur = conn.cursor()
    
    # get name of professor with specific phone number
    if intent == "prof_name_query_telephone":
        cur.execute("select title, first_name, last_name FROM PROF_INFO_TABLE where telephone = ? COLLATE NOCASE", (condition[0],))
        conn.commit()
        answer = cur.fetchall()
        
    # get name of professor with specific email
    elif intent == "prof_name_query_email": 
        cur.execute("select title, first_name, last_name FROM PROF_INFO_TABLE where email = ? COLLATE NOCASE", (condition[0],))
        conn.commit()
        answer = cur.fetchall()
        
    # get name of professor with specific office
    elif intent == "prof_name_query_office":
        cur.execute("select title, first_name, last_name FROM PROF_INFO_TABLE where office = ? COLLATE NOCASE", (condition[0],))
        conn.commit()
        answer = cur.fetchall()
        
    # get name of professor with specific research area
    elif intent == "prof_name_query_research_area":
        cur.execute("SELECT title, first_name, last_name FROM PROF_INFO_TABLE INNER JOIN PROF_RESEARCH_AREA_TABLE on PROF_RESEARCH_AREA_TABLE.prof_id = PROF_INFO_TABLE.prof_id WHERE research_area = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchone()
    
    # get name of professor with specific study
    elif intent == "prof_name_query_study":
        cur.execute("SELECT title, first_name, last_name FROM PROF_INFO_TABLE INNER JOIN PROF_STUDY_TABLE on PROF_STUDY_TABLE.prof_id = PROF_INFO_TABLE.prof_id WHERE study = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchone()
        
    # get last name of professor
    elif intent == "prof_name_query_lastname":
        cur.execute("select last_name FROM PROF_INFO_TABLE where first_name = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get first name of professor
    elif intent == "prof_name_query_firstname":
        cur.execute("select first_name FROM PROF_INFO_TABLE where last_name = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get phone number of professor
    elif intent == "prof_telephone_query_name":
        cur.execute("select telephone FROM PROF_INFO_TABLE where last_name =? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get email of professor
    elif intent == "prof_email_query_name":
        cur.execute("select email FROM PROF_INFO_TABLE where last_name =? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get office of professor
    elif intent == "prof_office_query_name":
        cur.execute("select office FROM PROF_INFO_TABLE where last_name =? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get research area of professor
    elif intent == "prof_research_area_query_name":
        cur.execute("SELECT research_area FROM PROF_RESEARCH_AREA_TABLE INNER JOIN PROF_INFO_TABLE on PROF_INFO_TABLE.prof_id = PROF_RESEARCH_AREA_TABLE.prof_id WHERE last_name = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # get study of professor
    elif intent == "prof_study_query_name":
        cur.execute("SELECT study FROM PROF_STUDY_TABLE INNER JOIN PROF_INFO_TABLE on PROF_INFO_TABLE.prof_id = PROF_STUDY_TABLE.prof_id WHERE last_name = ? COLLATE NOCASE", (condition,))
        conn.commit()
        answer = cur.fetchall()
        
    # generic conversation no retrieval necessary
    if intent in intent_generic_conversation:
        answer = "generic conversation"
    
    return answer

#### 6.4.2 Action Execution - Information Extraction and Storing - Update

In [12]:
def action_execution_update(intent, prof_name, new_info):
    
    #connect to database
    conn = sqlite3.connect("PROF_INFO_DB.db")
    cur = conn.cursor()
    
    # get phone number from input
    if intent == "extract_new_telephone":
        cur.execute("UPDATE PROF_INFO_TABLE SET telephone = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()

    # get email from input
    elif intent == "extract_new_email": 
        cur.execute("UPDATE PROF_INFO_TABLE SET email = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()
    
    # get office from input
    elif intent == "extract_new_office":
        cur.execute("UPDATE PROF_INFO_TABLE SET office = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()
    
    # get research area from input
    elif intent == "extract_new_research_area":
        cur.execute("INSERT INTO PROF_RESEARCH_AREA_TABLE (prof_id,Name,research_area) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (prof_name, prof_name, new_info,))
        conn.commit()
        
    # get study from input
    elif intent == "extract_new_study":
        #print("pog", condition[0])
        cur.execute("INSERT INTO PROF_STUDY_TABLE (prof_id,Name,study) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (prof_name, prof_name, new_info,))
        conn.commit()
    
    return 

#### 6.4.3 Action Execution Information Extraction and Storing - New Professor

In [13]:
# load NER model to identify names of new professor
nlp_name = spacy.load("en_core_web_md")

In [14]:
def action_execution_new_prof():
    
    # get name of new professor
    print("Great! What is the name of the new Professor?")
    inp = input("You: ")
    doc = nlp_name(inp)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            first_name = ent.text.split()[0]
            last_name = ent.text.split()[1]
        else:
            first_name = "Couldn't extract the name"
            last_name ="Couldn't extract the name"
    
    # get phone number
    print("What is the phone number")
    inp = input("You: ")
    
    re_number_1 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
    re_number_2 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{3}"
    re_number_3 = r"\D[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
    
    try:
        telephone = re.compile("(%s|%s|%s)" % (re_number_1, re_number_2, re_number_3)).findall(inp)      
    
    except: 
        telephone = "no number"
    
    # get email
    print("What is the email address?")
    inp = input("You: ")
    
    try:
        email = re.findall('\S+@\S+', inp)
    
    except:
        email = "no email"
    
    # get office
    print("In which office is the new professor sitting?")
    inp = input("You: ")
    
    re_office_1 = r"[A-Z, a-z].\d{1}.\d{2}"
    re_office_2 = r"[A-Z, a-z].\d{3}"
    
    try:
        office = re.compile("(%s|%s)" % (re_office_1, re_office_2)).findall(inp)
    
    except:
        ofiice = "no office"
    
    # get research areas
    print("What are the research areas of the new professor?")
    inp = input("You: ")
    
    doc = nlp_research_area(inp)
    for ent in doc.ents:
        if ent.label_ == "RESEARCH_AREA":
            research_area = ent.text  
        else:
            research_area = "Not found"
    
    # get study
    print("In what study is the new professor teaching?")
    inp = input("You: ")
    doc = nlp_study(inp)
    for ent in doc.ents:
        if ent.label_ == "STUDY":
            study = ent.text  
        else:
            study = "Not found"
      
    # connect to database
    conn = sqlite3.connect("PROF_INFO_DB.db")
    cur = conn.cursor()

    # insert new information into database
    cur.execute("INSERT INTO PROF_INFO_TABLE (prof_id, first_name,last_name, telephone, email, office) VALUES((SELECT MAX(prof_id) FROM PROF_INFO_TABLE)+1,?,?,?,?,?)", (first_name, last_name, telephone[0], email[0], office[0]))
    cur.execute("INSERT INTO PROF_RESEARCH_AREA_TABLE (prof_id, Name, research_area) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (last_name, last_name,research_area))
    cur.execute("INSERT INTO PROF_STUDY_TABLE (prof_id, Name, study) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (last_name, last_name,study))

    conn.commit()
    return 

### 6.5. Response Generation

Je nach ausgeführter Aktion sind unterschiedliche Antworten des Chatbots an den Benutzer notwendig. Demnach wurde die Response Generation für die Übersichtlichkeit ebenfalls aufgeteilt. Für jeden Intent wurden im Vorfeld eine Reihe von Antworten mit Platzhalter definiert und in einer json Datei gespeichert. Diese Datei muss zunächst geladen werden und die Antworten für jedes nach Intent in einem Dictionary gespeichert werden. Die Response Generation Funktionen nehmen zufällig eine passende Antwort, ersetzten die Platzhalter mit den richtigen Informationen und geben diese an den Benutzer zurück. 

#### 6.5.1 Response Generation - Information Retrieval

In [15]:
# open and load json file with responses and intents
with open("5. Response Generation/responses.json") as file:
    data = json.load(file)

In [16]:
# get the intent labels and the reponses from the file and save them in a dictionary
tags = [] #all intents in the json file
resp = []

for response in data["responses"]:

    if response["tag"] not in tags:
        tags.append(response["tag"])

    if response["sentence"] not in resp:
        resp.append(response["sentence"])
        
responses_dict = dict(zip(tags, resp))

In [17]:
def response_generation_retrieval(intent, answer, condition):
    
    if intent == "greeting":
        responses = responses_dict["greeting"]
        final_response = random.choice(responses)
        
    elif intent == "greeting_response":
        responses = responses_dict["greeting_response"]
        final_response = random.choice(responses)
        
    elif intent == "courtesy_greeting":
        responses = responses_dict["courtesy_greeting"]
        final_response = random.choice(responses)
        
    elif intent == "courtesy_greeting_response":
        responses = responses_dict["courtesy_greeting_response"]
        final_response = random.choice(responses)
        
    elif intent == "real_name_query":
        responses = responses_dict["real_name_query"]
        final_response = random.choice(responses)
        
    elif intent == "goodbye":
        responses = responses_dict["goodbye"]
        final_response = random.choice(responses)
        
    elif intent == "task_response":
        responses = responses_dict["task_response"]
        final_response = random.choice(responses)     
        
    elif intent == "prof_name_query_lastname":
        responses = responses_dict["prof_name_query_lastname"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0][0]) 
               
    elif intent == "prof_name_query_firstname":
        responses = responses_dict["prof_name_query_firstname"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0][0]) 
       

    elif intent == "prof_name_query_telephone":
        responses = responses_dict["prof_name_query_telephone"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition[0]) 
        final_response = new_reponse.replace("_answer_", answer[0][0] + " " + answer[0][1] + " " + answer[0][2]) 
        
    elif intent == "prof_name_query_email": 
        responses = responses_dict["prof_name_query_email"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition[0]) 
        final_response = new_reponse.replace("_answer_", answer[0][0] + " " + answer[0][1] + " " + answer[0][2])       
        
    elif intent == "prof_name_query_office":
        responses = responses_dict["prof_name_query_office"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition[0]) 
        final_response = new_reponse.replace("_answer_", answer[0][0] + " " + answer[0][1] + " " + answer[0][2])      
        
    elif intent == "prof_name_query_research_area":
        responses = responses_dict["prof_name_query_research_area"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0] + " " + answer[1] + " " + answer[2])  
        
    elif intent == "prof_name_query_study":
        responses = responses_dict["prof_name_query_study"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0] + " " + answer[1] + " " + answer[2])  
        
    elif intent == "prof_telephone_query_name":
        responses = responses_dict["prof_telephone_query_name"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0][0])  
       
    elif intent == "prof_email_query_name":
        responses = responses_dict["prof_email_query_name"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0][0])  
        
    elif intent == "prof_office_query_name":
        responses = responses_dict["prof_office_query_name"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", answer[0][0])  
        
    elif intent == "prof_research_area_query_name":
        new_answer = ""
        for i in range(len(answer)):
            new_answer += answer[i][0] + ", "
            
        responses = responses_dict["prof_research_area_query_name"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", new_answer)  
        
    elif intent == "prof_study_query_name":
        new_answer = ""
        for i in range(len(answer)):
            new_answer += answer[i][0] + ", "
        
        responses = responses_dict["prof_study_query_name"]
        response = random.choice(responses)
        new_reponse = response.replace("_condition_", condition) 
        final_response = new_reponse.replace("_answer_", new_answer)  
        
    return final_response

#### 6.5.1 Response Generation - Information Extraction and Storing

In [18]:
def response_generation_update(intent, condition, prof_name):
           
    if intent == "extract_new_telephone":
        response = "I changed the telephone number. Thanks"
              
    elif intent == "extract_new_email":
        response = "I changed the email address. Thanks"
              
    elif intent == "extract_new_office":
        response = "I changed the office number. Thanks"
              
    elif intent == "extract_new_research_area":
        response = "I added the new research area. Thanks"
              
    elif intent == "extract_new_study":
        response = "I added the new study. Thanks"
        
    return response

### 6.7. Chat

Hier wird die finale Chatbot Funktion definiert, die alle zuvor implementierten Funktionen nutzt. Je nach Intent werden die passenden Funktion ausgeführt.

In [19]:
def chat():
    log = []
    print("Start talking with me!(type quit to stop):")
    while True:
        inp = input("You: ")
        log.append(inp) #saves all input of user in list
        if inp.lower() == "quit":
            print("Goodbye :)")
            break
        
        # Intent Classification
        try:
            results = model.predict([bag_of_words(inp, words)])[0] #output is just a probability for each label
            results_index = numpy.argmax(results) #index of greatest value
            intent = labels[results_index] #output is the most probable label
            #print(intent)
            
            
            # Information Extraction and Storing - New professor                
            if intent == "extract_new_prof":
                # Action Execution
                action_execution_new_prof()
                # Response Generation
                response = "Thank you. I saved the new information in the database"
                print(response)
                
            # Information Extraction and Storing - Update Information
            if "new" in intent:
                # Slot Filling
                prof_name, condition = slot_filling_update(inp, intent)
                # Action Execution
                action_execution_update(intent, prof_name, condition)
                # Response Generation
                response = response_generation_update(intent, condition, prof_name)
                print(response)

            # Information Retrieval 
            else:
                # Slot Filling
                condition = slot_filling_retrieval(inp, intent)
                # Action Execution
                answer = action_execution_retrieval(intent, condition)
                # Response Generation
                final_response = response_generation_retrieval(intent, answer, condition)
                print(final_response)
                
        # Fallback
        except:
            print("Sorry. I dont't understand. Can you say it with other words?")

    return log

In [20]:
chat()

Start talking with me!(type quit to stop):
You: hello
Hi
You: how are you
I'm good. Thanks
You: whats your name
I am KG-Bot
You: what can you do 
Sorry. I dont't understand. Can you say it with other words?
You: what is your task?
I can help you getting information about HHN Professors
You: What is the email of herr lanquillon
The email of Lanquillon is carsten.lanquillon@hs-heilbronn.de
You: there is a new professor
Great! What is the name of the new Professor?
You: peter blume 
What is the phone number
You: +00 000 0000 000
What is the email address?
You: peter@web.de
In which office is the new professor sitting?
You: t,333
What are the research areas of the new professor?
You: TENNIS
In what study is the new professor teaching?
You: TENNIS
Sorry. I dont't understand. Can you say it with other words?
You: a
Bye
You: a
Goodbye
You: quit
Goodbye :)


['hello',
 'how are you',
 'whats your name',
 'what can you do ',
 'what is your task?',
 'What is the email of herr lanquillon',
 'there is a new professor',
 'a',
 'a',
 'quit']